# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine  
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('MessagesCategories', engine)  
X = df['message'].values
Y = df.iloc[:, 4:].values
#X = np.delete(X,7549, axis = 0)
#Y = np.delete(Y,7549, axis = 0)

In [5]:
print(X.shape, Y.shape)

(26216,) (26216, 36)


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text, url_place_holder_string="urlplaceholder"):
     # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    hashtag_regex = '#[a-zA-Z0-9_]+'
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Extract all the hashtags from the provided text 
    detected_tags = re.findall(hashtag_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
    
    # Replace url with a url placeholder string
    for detected_tag in detected_tags:
        text = text.replace(detected_tag, detected_tag[1:])

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    clean_tokens = [ w for w in clean_tokens if re.search(r'[a-zA-Z]+', w)]
    return clean_tokens

In [7]:
for x in X[:10]:
    print(tokenize(x),"\n")

['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

['looking', 'for', 'someone', 'but', 'no', 'name'] 

['un', 'report', 'leogane', 'destroyed', 'only', 'hospital', 'st.', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 

['information', 'about', 'the', 'national', 'palace-'] 

['storm', 'at', 'sacred', 'heart', 'of', 'jesus'] 

['please', 'we', 'need', 'tent', 'and', 'water', 'we', 'are', 'in', 'silo', 'thank', 'you'] 

['i', 'would', 'like', 'to', 'receive', 'the', 'message', 'thank', 'you'] 

['i', 'am', 'in', 'croix-des-bouquets', 'we', 'have', 'health', 'issue', 'they', 'worker', 'are', 'in', 'santo', 'an', 'area', 'in', 'croix-des-bouquets'] 



In [8]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        tokens = tokenize(text)
        pos_tags = nltk.pos_tag(tokens)
        #print(pos_tags)
        try:
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            return False
        except:
            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [9]:
def starting_verb(text):
        tokens = tokenize(text)
        pos_tags = nltk.pos_tag(tokens)
        #print(pos_tags)
        try:
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            return False
        except:
            return False

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer()),
            ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)



In [13]:
# Print classification report on test data
print(classification_report(np.hstack(y_test),np.hstack(y_prediction_test)))

             precision    recall  f1-score   support

          0       0.96      0.98      0.97    215254
          1       0.77      0.59      0.67     20635
          2       0.43      0.18      0.26        55

avg / total       0.94      0.95      0.95    235944



### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('count_vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8b8531b378>, vocabulary=None)),
  ('tfidf_transformer',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'count_vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_f

In [20]:
parameters = {
    'classifier__estimator__n_estimators': [50, 100],
    'classifier__estimator__learning_rate': [1.0, 0.3],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=3, verbose=2)

In [16]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50 
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50 
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50 
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=100 
[CV]  classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50, total= 7.1min
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=100 
[CV]  classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50, total= 7.1min
[CV] classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=100 
[CV]  classifier__estimator__learning_rate=1.0, classifier__estimator__n_estimators=50, total= 7.1min
[CV] classifier__estimator__learning_rate=0.3, classifier__estimator__n_estimators=50 
[CV]  classifier__esti

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 33.5min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 33.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 ...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'classifier__estimator__n_estimators': [50, 100], 'classifier__estimator__learning_rate': [1.0, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [17]:
#finding the best paramesters based on grip search
print(cv.best_params_)

#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

{'classifier__estimator__learning_rate': 0.3, 'classifier__estimator__n_estimators': 100}
Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 ...ator=None,
          learning_rate=0.3, n_estimators=100, random_state=None),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_prediction_train = optimised_model.predict(X_train)
y_prediction_test = optimised_model.predict(X_test)

# Print classification report on test data
print(classification_report(np.hstack(y_test),np.hstack(y_prediction_test)))

             precision    recall  f1-score   support

          0       0.96      0.99      0.97    215254
          1       0.80      0.55      0.65     20635
          2       0.00      0.00      0.00        55

avg / total       0.94      0.95      0.94    235944



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
import pickle

In [22]:
with open('model.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.